In [18]:
# Get variance explained for 1) age-sex adjusted phenotypes 2) age-sex-bmi adjusted phenotypes and 3) prs scores

import pickle

demo = pickle.load(open('/home/anton/Documents/Tulane/Research/ImageNomer/data/UKB2/demographics.pkl', 'rb'))

print(list(demo.keys()))

['f_22001_0_0', 'f_21003_3_0', 'f_21003_2_0', 'f_21000_0_0', 'CaAfCarDiff', 'CaIndDiff', 'CaChiDiff', 'f_21000_0_0_comb', 'longitudinal', '22438-0.0', '22828-0.0', '26201-0.0', '26202-0.0', '26203-0.0', '26204-0.0', '26205-0.0', '26206-0.0', '26207-0.0', '26208-0.0', '26209-0.0', '26210-0.0', '26211-0.0', '26212-0.0', '26213-0.0', '26214-0.0', '26215-0.0', '26216-0.0', '26217-0.0', '26218-0.0', '26219-0.0', '26220-0.0', '26221-0.0', '26222-0.0', '26223-0.0', '26224-0.0', '26225-0.0', '26226-0.0', '26227-0.0', '26228-0.0', '26229-0.0', '26231-0.0', '26232-0.0', '26233-0.0', '26234-0.0', '26235-0.0', '26236-0.0', '26237-0.0', '26238-0.0', '26239-0.0', '26240-0.0', '26241-0.0', '26242-0.0', '26243-0.0', '26244-0.0', '26245-0.0', '26246-0.0', '26247-0.0', '26248-0.0', '26249-0.0', '26250-0.0', '26251-0.0', '26252-0.0', '26253-0.0', '26254-0.0', '26255-0.0', '26256-0.0', '26257-0.0', '26258-0.0', '26259-0.0', '26260-0.0', '26261-0.0', '26262-0.0', '26263-0.0', '26264-0.0', '26265-0.0', '262

In [19]:
# Load all fc

from pathlib import Path
import re
import numpy as np

fcdict = {}
i = 0

for file in Path('/home/anton/Documents/Tulane/Research/ImageNomer/data/UKB2/conn/').iterdir():
    match = re.match('(.*)_task-20227-2_fc.npy', file.name)
    if match:
        sub = match.group(1)
        p = np.load(f'/home/anton/Documents/Tulane/Research/ImageNomer/data/UKB3/conn/{sub}_task-20227-2_fc.npy')
        fcdict[int(sub)] = p
        i += 1
        if i % 10000 == 0:
            print(f'Done {i}')

print('Done')

Done 10000
Done 20000
Done 30000
Done 40000
Done


In [20]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
import random

def rmse(a, b):
    return np.mean((a-b)**2)**0.5

bestdct = dict()

for key in demo.keys():
    if not re.match('262.*-0.0', key):
        continue
    print(key, len(demo[key]))
    fc = []
    var = []
    for sub in demo[key].keys():
        # Sample only
        if len(demo[key]) > 10000 and random.random() < 0.6:
            continue
        if sub in fcdict:
            p = fcdict[sub]
            v = demo[key][sub]
            fc.append(p)
            var.append(v)
    fc = np.stack(fc)
    var = np.stack(var)
    best = 0
    for alpha in [1]:
        xtr, xt, ytr, yt = train_test_split(fc, var, train_size=0.8)
        reg = Ridge(alpha=alpha).fit(xtr, ytr)
        yhat = reg.predict(xtr)
        loss = rmse(yhat, ytr)
        totvar = np.std(ytr)
        r2 = (totvar-loss)/totvar
        print(alpha, loss, totvar-loss, totvar, r2)
        if r2 > best:
            best = r2
    print(best)
    bestdct[key] = best

26201-0.0 39520
1 0.058286103247447194 7.9650955539883945 8.023381657235841 0.9927354691902408
0.9927354691902408
26202-0.0 39520


KeyboardInterrupt: 

In [18]:
for key, val in bestdct.items():
    if val > 0.005:
        print(key, val)

26201-0.0 0.027735306103147093
26240-0.0 0.008907846682085708
26241-0.0 0.021232905991657107
26245-0.0 0.007423388034629044


In [19]:
# Save R2 values

pickle.dump(bestdct, open('/home/anton/Documents/Tulane/Research/ImageNomer/data/UKB2/bestr2.pkl', 'wb'))

print('Done')

Done


In [17]:
# View saved R^2
# Note when viewing sex or age or BMI these have not been regressed from themselves

import pickle

d = pickle.load(open('/home/anton/Documents/Tulane/Research/ImageNomer/data/UKB4/bestr2_agesexbmi.pkl', 'rb'))

for key,val in d.items():
    print(key, val)

f_21003_2_0 0.23840433154417232
f_31_0_0 0.36999887854324226
f_46_0_0 0.19658489584095515
f_46_0_0_adj 0.0027937442343281566
f_47_0_0 0.15956815749546768
f_47_0_0_adj 0.0022238737730122136
f_48_0_0 0.1885108360618104
f_48_0_0_adj 0.006605642411651328
f_49_0_0 0.07339709842316784
f_49_0_0_adj 0.012115295150287942
f_50_0_0 0.2168036006158694
f_50_0_0_adj 0.03489807810799713
f_93_0_0 0.04475612554962355
f_93_0_0_adj 0.00848600312286178
f_94_0_0 0.015162901010549146
f_94_0_0_adj 0.02828473678878149
f_95_0_0 0.006588387054589898
f_95_0_0_adj 0
f_102_0_0 0.02017237548369511
f_102_0_0_adj 0.01229482305680149
f_134_0_0 0.00022735323714282666
f_134_0_0_adj 0
f_135_0_0 0.015603662095258796
f_135_0_0_adj 0
f_136_0_0 0.0035068280002126554
f_136_0_0_adj 0.0004151017129409106
f_137_0_0 0.015822740042121874
f_137_0_0_adj 0.0051910667169391975
f_738_0_0 0.00853122319826495
f_738_0_0_adj 0.00359803637527341
f_864_0_0 0
f_864_0_0_adj 0.0007206621360052761
f_874_0_0 0
f_874_0_0_adj 0.001255382355923128
f

In [15]:
# Get fc corr of 7,500 subject subset for standard principal component of PRS

def corr(ps, v):
    y = v - np.mean(v)
    x = ps - np.mean(ps, axis=0, keepdims=True)
    xx = np.einsum('na,na->a', x, x)
    yy = np.einsum('n,n->', y, y)
    xy = np.einsum('na,n->a', x, y)
    rho = xy/np.sqrt(xx*yy)
    rhox = np.einsum('a,na->n', rho, ps)
    return np.corrcoef(rhox, v)[0,1]

ps = []
vs = []
for sub,v in demo['26201-0.0'].items():
    p = fcdict[sub]
    vs.append(v)
    ps.append(p)
    if len(ps) > 7500:
        break

corr(np.stack(ps), np.stack(vs))

np.float64(0.08606533565414666)